In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import time

import CustomHyperModel
import CustomMetrics
import EnergyPricesLibrary as Ep

from kerastuner.tuners import BayesianOptimization

%load_ext autoreload
%autoreload 2

In [2]:
def make_predictions(model,scaler_y,trainX,trainY,testX,testY,n_steps_out,len_output_features):
    
    # make predictions
    trainPredict = model.predict(trainX)
    trainPredict = trainPredict.reshape(trainPredict.shape[0]*n_steps_out,len_output_features)
    testPredict  = model.predict(testX)
    testPredict  = testPredict.reshape(testPredict.shape[0]*n_steps_out,len_output_features)
    
    # invert predictions
    trainPredict = scaler_y.inverse_transform(trainPredict)
    trainY_ = scaler_y.inverse_transform(trainY.reshape(trainY.shape[0]*n_steps_out,len_output_features))
    
    testPredict = scaler_y.inverse_transform(testPredict)
    testY_ = scaler_y.inverse_transform(testY.reshape(testY.shape[0]*n_steps_out,len_output_features))
        
    return trainPredict,trainY_,testPredict,testY_

def get_metrics(trainY,trainPredict,testY,testPredict):
    
    trainMAPE  = Ep.MAPE(trainPredict,trainY)
    testMAPE  = Ep.MAPE(testPredict,testY)
    
    train_sMAPE  = Ep.sMAPE(trainY,trainPredict)
    test_sMAPE  = Ep.sMAPE(testY,testPredict)
    
    return trainMAPE,testMAPE,train_sMAPE,test_sMAPE

# DATASET

## Load Dataset

In [3]:
data_diaria_path = os.path.join('..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Diaria.xlsx')
data_diaria = pd.read_excel(data_diaria_path)
data_diaria = data_diaria.set_index('Fecha')

In [4]:
nombre_series_diaria = data_diaria.columns.values

In [5]:
precio_bolsa_path = os.path.join('..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Precio_Bolsa.xlsx')
precio_bolsa = pd.read_excel(precio_bolsa_path)
precio_bolsa = precio_bolsa.set_index('Fecha')

In [6]:
data_diaria.shape, precio_bolsa.shape

((7395, 119), (177480, 1))

## Build Window

In [7]:
d = 'All'
start_date_train = '2000-02-01'
start_date_val = '2020-01-01'
start_date_test = '2020-04-01'
end_date_test = '2020-05-01'
n_steps_out=24
output_columns = ['$kWh']

In [8]:
n_steps_in = 15
overlap = 1
inputs_columns = nombre_series_diaria

len_input_features = len(inputs_columns)
len_output_features = len(output_columns)

results = Ep.SplitTimeseriesMultipleTimesBackAhead_differentTimes(df_x=data_diaria,
                                                                  df_y=precio_bolsa,
                                                                  day=d,
                                                                  start_date_train=start_date_train,start_date_val=start_date_val,
                                                                  start_date_test=start_date_test,end_date_test=end_date_test,
                                                                  n_steps_out=n_steps_out,n_steps_in=n_steps_in,
                                                                  overlap=overlap,input_features=inputs_columns,
                                                                  output_features=output_columns)

trainX_D,trainY_D,valX_D,valY_D,testX_D,testY_D,scaler_D_x,scaler_D_y,dataset_x_D, dataset_y_D = results

In [9]:
print(trainX_D.shape,trainY_D.shape,valX_D.shape,valY_D.shape,testX_D.shape,testY_D.shape)

(7259, 15, 119) (7259, 24, 1) (91, 15, 119) (91, 24, 1) (30, 15, 119) (30, 24, 1)


In [10]:
"""
log_dir = os.path.join('logs','Optimización')
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
callback_tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                          histogram_freq=2,
                                                          write_graph=False,
                                                          update_freq='epoch
"""
    
callback_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                          factor=0.1,
                                                          min_lr=1e-5,
                                                          patience=5,
                                                          verbose=1)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=10,
                                                  mode='min')

#callbacks = [callback_tensorboard,callback_reduce_lr,early_stopping]
callbacks = [callback_reduce_lr,early_stopping]

In [11]:
INPUT_SHAPE = (n_steps_in,len_input_features)

arquitectura31 = CustomHyperModel.Arquitectura31(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura32 = CustomHyperModel.Arquitectura32(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura33 = CustomHyperModel.Arquitectura33(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura34 = CustomHyperModel.Arquitectura34(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura35 = CustomHyperModel.Arquitectura35(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura36 = CustomHyperModel.Arquitectura36(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura37 = CustomHyperModel.Arquitectura37(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura38 = CustomHyperModel.Arquitectura38(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)

arquitectura39 = CustomHyperModel.Arquitectura39(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura40 = CustomHyperModel.Arquitectura40(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura41 = CustomHyperModel.Arquitectura41(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura42 = CustomHyperModel.Arquitectura42(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura43 = CustomHyperModel.Arquitectura43(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura44 = CustomHyperModel.Arquitectura44(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura45 = CustomHyperModel.Arquitectura45(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura46 = CustomHyperModel.Arquitectura46(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)

In [12]:
arq_list = [arquitectura31,arquitectura32,arquitectura33,arquitectura34,
            arquitectura35,arquitectura36,arquitectura37,arquitectura38,
            arquitectura39,arquitectura40,arquitectura41,arquitectura42,
            arquitectura43,arquitectura44,arquitectura45,arquitectura46]
"""
arq_list = [arquitectura35,arquitectura36,arquitectura37,arquitectura38,
            arquitectura39,arquitectura40,arquitectura41,arquitectura42,
            arquitectura43,arquitectura44,arquitectura45,arquitectura46]
"""

'\narq_list = [arquitectura35,arquitectura36,arquitectura37,arquitectura38,\n            arquitectura39,arquitectura40,arquitectura41,arquitectura42,\n            arquitectura43,arquitectura44,arquitectura45,arquitectura46]\n'

In [13]:
arq_idx = 31
arq_best_models = dict()

for arq in arq_list:
    
    bayesian_tuner = BayesianOptimization(
        arq,
        objective='val_loss',
        num_initial_points=1,
        max_trials=10,
        directory='my_dir',
        project_name=str(arq_idx)
    )
    
    # Overview of the task
    bayesian_tuner.search_space_summary()
    
    # Performs the hyperparameter tuning
    search_start = time.time()
    bayesian_tuner.search(x=trainX_D,y=trainY_D,
                      epochs=200,
                      validation_data=(valX_D,valY_D),
                      callbacks=callbacks)
    search_end = time.time()
    elapsed_time = search_end - search_start
    
    print('Tiempo Total Transcurrido {}'.format(elapsed_time))
        
    dict_key = 'Arquitectura {}'.format(arq_idx)

    arq_best_models[dict_key] = dict()
    bs_model = bayesian_tuner.oracle.get_best_trials(1)[0]
    
    model = bayesian_tuner.get_best_models(num_models=1)[0]
    
    trainPredict,trainY_true,testPredict,testY_true = make_predictions(model,scaler_D_y,trainX_D,trainY_D,valX_D,valY_D,
                                                                       n_steps_out,len_output_features)
    
    trainMAPE,testMAPE,train_sMAPE,test_sMAPE = get_metrics(trainY_true,trainPredict,testY_true,testPredict)

    arq_best_models[dict_key]['Score'] = bs_model.score
    arq_best_models[dict_key]['Tiempo Scaneo'] = elapsed_time
    arq_best_models[dict_key]['Mape Train'] = trainMAPE
    arq_best_models[dict_key]['Mape Test'] = testMAPE
    arq_best_models[dict_key]['sMape Train'] = train_sMAPE
    arq_best_models[dict_key]['sMape Test'] = test_sMAPE

    if bs_model.hyperparameters.values:
        for hp, value in bs_model.hyperparameters.values.items():
            arq_best_models[dict_key][hp] = value
    
    arq_idx += 1

Trial 10 Complete [00h 01m 42s]
val_loss: 1316.1876220703125

Best val_loss So Far: 507.40264892578125
Total elapsed time: 00h 08m 24s
INFO:tensorflow:Oracle triggered exit
Tiempo Total Transcurrido 504.8873836994171


In [14]:
import json
with open('BestModels-D15-V7.json', 'w') as outfile:
    json.dump(arq_best_models, outfile)

In [15]:
arq_best_models

{'Arquitectura 31': {'Score': 679.4625854492188,
  'Tiempo Scaneo': 0.0009965896606445312,
  'Mape Train': 0.2153613907896397,
  'Mape Test': 0.22310306078787528,
  'sMape Train': 22.363077429410215,
  'sMape Test': 23.104376817486266,
  'lstm_units_layer_1': 384,
  'kernel_regularizer_layer_1': 0.0375,
  'dropout_regularizer_layer_1': 0.54,
  'lstm_units_layer_2': 320,
  'kernel_regularizer_layer_2': 0.0675,
  'dropout_regularizer_layer_2': 0.18,
  'dense_layer': 96,
  'dense_layer_activation': 'relu',
  'dropout_dense': 0.18,
  'learning_rate': 0.0012535163155377037},
 'Arquitectura 32': {'Score': 522.8075561523438,
  'Tiempo Scaneo': 0.0009913444519042969,
  'Mape Train': 0.22170822896047487,
  'Mape Test': 0.19718353868768015,
  'sMape Train': 21.07640333651077,
  'sMape Test': 16.71404977288025,
  'gru_units_layer_1': 192,
  'kernel_regularizer_layer_1': 0.0375,
  'dropout_regularizer_layer_1': 0.27,
  'gru_units_layer_2': 384,
  'kernel_regularizer_layer_2': 0.075,
  'dropout_reg